In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/ns1254/mimicgen/envs/robomimic/robomimic/scripts/setup_macros.py
)


### create new file taking demo from multiple files.

In [2]:
hdf5_files_groups ={
    "/home/ns1254/data_franka/bellpepper_oma/ola_114.hdf5": ["good"], 
    "/home/ns1254/data_franka/bellpepper_oma/marzan_73.hdf5": ["good"],
    "/home/ns1254/data_franka/bellpepper_oma/akash_100.hdf5": ["good"]
}

In [3]:
for filepath, groups in hdf5_files_groups.items():
    f = h5py.File(filepath, "r")
    if len(groups)<1:
        demos = list(f["data"].keys())      
    elif len(groups)==1:
        demos = [b.decode('utf-8') for b in f["mask"][groups[0]]]
    else:
        print('---------not implemented-------------')
    
    print(f"filepath: {filepath} groups: {groups} #demo: {len(demos)}")
    f.close()
    

filepath: /home/ns1254/data_franka/bellpepper_oma/ola_114.hdf5 groups: ['good'] #demo: 94
filepath: /home/ns1254/data_franka/bellpepper_oma/marzan_73.hdf5 groups: ['good'] #demo: 51
filepath: /home/ns1254/data_franka/bellpepper_oma/akash_100.hdf5 groups: ['good'] #demo: 35


In [4]:
def copy_group(src_group, dest_group):
    # Copy attributes
    for attr_name, attr_value in src_group.attrs.items():
        dest_group.attrs[attr_name] = attr_value

    for key, item in src_group.items():
        if isinstance(item, h5py.Group):
            # Create a new group in the destination and recursively copy contents
            new_group = dest_group.create_group(key)
            copy_group(item, new_group)
        elif isinstance(item, h5py.Dataset):
            # Copy datasets
            dataset = dest_group.create_dataset(key, data=item[...])
            # Copy attributes for the dataset
            for attr_name, attr_value in item.attrs.items():
                dataset.attrs[attr_name] = attr_value


In [5]:
def copy_demos(f_src, f_dest, demos2copy, next_id):
    new_demo_names=[]
    i=next_id
    for demo_name in tqdm.tqdm(demos2copy):
        demo=f_src['data'][demo_name]  

        demo_name_new="demo_"+str(i) 
        new_demo=f_dest["data"].create_group(demo_name_new) 
        copy_group(demo, new_demo) 

        new_demo_names.append(demo_name_new)
        i = i+1

    return i , new_demo_names

In [6]:
new_file_path = "/home/ns1254/data_franka/bellpepper_oma/combined_oma_good.hdf5"

fc = h5py.File(new_file_path, "w")
fc.create_group("data")
fc.create_group("mask")

<HDF5 group "/mask" (0 members)>

In [7]:
next_id = 0
for filepath, groups in hdf5_files_groups.items():
    f_src = h5py.File(filepath, "r")
    if len(groups)<1:
        demos2copy = list(f_src["data"].keys())      
    elif len(groups)==1:
        demos2copy = [b.decode('utf-8') for b in f_src["mask"][groups[0]]]
    else:
        print('---------not implemented-------------')
        print('ignored-------------')
        continue
    
    print(f"filepath: {filepath} groups: {groups} #demo: {len(demos2copy)}")

    #copy attributes from first file
    if next_id==0:
        print('First: copying attributes from the first file.')
        for attr_name, attr_value in f_src.attrs.items():
            fc.attrs[attr_name] = attr_value

    print('copying demos...')
    next_id , new_demo_names=copy_demos(f_src, fc, demos2copy, next_id)

    src_filename=os.path.basename(filepath).split(".")[0]
    
    print(f"creating filter key: {src_filename} with {len(new_demo_names)} demos")
    demos= np.array(new_demo_names, dtype='S8')   
    filter_keys=sorted([elem for elem in demos]) 
    filter_lengths = create_hdf5_filter_key(hdf5_path=new_file_path, demo_keys=filter_keys, key_name=src_filename)

    print('')
    f.close()

filepath: /home/ns1254/data_franka/bellpepper_oma/ola_114.hdf5 groups: ['good'] #demo: 94
First: copying attributes from the first file.
copying demos...


100%|██████████| 94/94 [00:06<00:00, 13.77it/s]


creating filter key: ola_114 with 94 demos

filepath: /home/ns1254/data_franka/bellpepper_oma/marzan_73.hdf5 groups: ['good'] #demo: 51
copying demos...


100%|██████████| 51/51 [00:06<00:00,  7.40it/s]


creating filter key: marzan_73 with 51 demos

filepath: /home/ns1254/data_franka/bellpepper_oma/akash_100.hdf5 groups: ['good'] #demo: 35
copying demos...


100%|██████████| 35/35 [00:02<00:00, 13.93it/s]

creating filter key: akash_100 with 35 demos



In [9]:
for key in fc['mask'].keys():
    print(key, fc['mask'][key].shape)

akash_100 (35,)
marzan_73 (51,)
ola_114 (94,)


In [10]:
demos =  list(fc["data"].keys())     
len(demos)

180

In [11]:
fc.close()